In [1]:
import os, cv2, sys, time, numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import re
import subprocess, pickle

In [33]:
def load_frames(input_path, num_frames):
    # Carica i frame e li converte in un array 2D per la PCA.
    frames = []
    for i in range(num_frames):
        num = str(i).zfill(3)
        frame_path = os.path.join(input_path, f"Frame_{num}.png")
        frame = cv2.imread(frame_path)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame.flatten())  # Appiattisci ogni frame in un vettore
    return np.array(frames)

def apply_pca_to_dataset(frames, n_components, original_shape):
    # Applica la PCA al dataset di frame.
    pca = PCA(n_components = n_components)
    frames_compressed = pca.fit_transform(frames)  # Compressione
    # frames_reconstructed = pca.inverse_transform(frames_compressed)  # Ricostruzione
    
    serialize_pca(pca, original_shape, frames_compressed, '../results/PCA/Dice/')


def save_reconstructed_frames(output_path, frames_compressed, original_shape, components, mean):
    reconstructed_data = np.dot(frames_compressed, components) + mean
    
    # Salva i frame ricostruiti come immagini.
    os.makedirs(output_path, exist_ok=True)
    for i, frame in enumerate(reconstructed_data):
        num = str(i).zfill(3)
        # Ricostruisci la forma originale
        frame_reshaped = frame.reshape(original_shape).astype(np.uint8)
        frame_path = os.path.join(output_path, f"Frame_{num}.png")
        cv2.imwrite(frame_path, cv2.cvtColor(frame_reshaped, cv2.COLOR_RGB2BGR))
        
def serialize_pca(pca, original_shape, compressed_data, output_path):
    # Salva il modello PCA e i dati compressi in un file.
    data_to_save = {
        'compressed_data': compressed_data,
        'original_shape': original_shape
    }
    with open(output_path + 'dice_pca.pkl', 'wb') as f:
        pickle.dump(data_to_save, f)
        
    np.savez_compressed(output_path + 'components.npz', pca.components_)
        
    
        
def deserialize_pca(input_path):
    # Carica il modello PCA e i dati compressi da un file.
    with open(input_path, 'rb') as f:
        data = pickle.load(f)
    return data['compressed_data'], data['original_shape'], data['components'], data['mean']




In [35]:
# encoder
n_components = 25 # Numero di componenti principali desiderati

# Carica i frame
sample_frame = cv2.imread(os.path.join("../datasets/Dice/", "Frame_000.png"))
original_shape = sample_frame.shape
frames = load_frames("../datasets/Dice/", 25)

# Applica la PCA al dataset
apply_pca_to_dataset(frames, n_components, original_shape)

In [36]:
# decoder
frames_compressed, original_shape, components, mean = deserialize_pca('../results/PCA/Dice/dice_pca.pkl')
print(type(components))
#save_reconstructed_frames('../results/PCA/Dice/', frames_compressed, original_shape, components, mean)

KeyError: 'components'